In [22]:
import json
import numpy as np
import pandas as pd
from glob import glob
from sklearn.metrics import classification_report, f1_score

In [23]:
strats = ["igcngru_features", "idarkvec", "features"]
DATA_SOURCE = "honeypot"

In [24]:
data_dir = f"../data/2022/input/stacking_predictions/out/k3/{DATA_SOURCE}/test"
days = [ f.split('/')[-1].split('_')[1] for f in glob(f"{data_dir}/idarkvec_*_fold00.csv")]
days.sort()
days

['20221021',
 '20221022',
 '20221023',
 '20221024',
 '20221025',
 '20221026',
 '20221027',
 '20221028',
 '20221029',
 '20221030',
 '20221031']

In [25]:
with open("../data/2022/input/skf/stratification/stratification.json", 'r') as fd:
    splits = json.load(fd)

In [26]:
source_to_remove = "honeypot" if DATA_SOURCE == "darknet" else "darknet"
print(source_to_remove)

darknet


In [27]:
df = pd.read_csv(f"../data/2022/input/stacking_predictions/out/k3/{DATA_SOURCE}/test/idarkvec_20221021_fold00.csv")

In [28]:
df = df[df.y_true != "unknown"].copy()

In [29]:
df.head()

,Unnamed: 0,src_ip,censys,driftnet,internetcensus,intrinsec,ipip,mirai,onyphe,rapid7,securitytrails,shadowserver,shodan,u_mich,unk_bruteforcer,unk_exploiter,unk_spammer,unknown,y_true
0,17639,117.187.173.104,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,unk_spammer
1,17640,222.185.146.149,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,mirai
2,17641,117.196.109.167,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.333333,mirai
3,17642,170.106.173.40,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,unk_spammer
4,17643,123.130.210.226,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,mirai


In [30]:
prob_cols = [
    "censys",
    "driftnet",
    "internetcensus",
    "intrinsec",
    "ipip",
    "mirai",
    "onyphe",
    "rapid7",
    "securitytrails",
    "shadowserver",
    "shodan",
    "u_mich",
    "unk_bruteforcer",
    "unk_exploiter",
    "unk_spammer",
    "unknown"
]
prob_cols.sort()

In [31]:
metrics = {}
for strat in strats:
    metrics[strat] = {}
    for day in days:
        for fold in np.arange(10):
            df = pd.read_csv(f"{data_dir}/{strat}_{day}_fold0{fold}.csv")
            ips_to_remove = splits[day][source_to_remove][fold][1]
            df = df[~df.src_ip.isin(ips_to_remove)]
            preds = df[prob_cols].values.argmax(axis=1)
            preds = [prob_cols[c] for c in preds]
            y_true = df.y_true.values
            report = classification_report(
                y_true,
                preds,
                labels=np.unique(y_true),
                output_dict=True,
                zero_division=0,
            )
            for c in prob_cols:
                if c in report:
                    if c not in metrics[strat]:
                        metrics[strat][c] = []
                    metrics[strat][c].append(report[c]["f1-score"])

In [32]:
scores = pd.DataFrame(metrics)
scores

,igcngru_features,idarkvec,features
censys,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.8, ...","[0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, ..."
driftnet,"[0.4210526315789474, 0.4210526315789474, 0.421...","[0.9090909090909091, 0.9090909090909091, 0.909...","[0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, ..."
ipip,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
mirai,"[0.9688073394495412, 0.9688073394495412, 0.966...","[0.9617486338797814, 0.9652650822669104, 0.961...","[0.9799635701275047, 0.9799635701275047, 0.981..."
onyphe,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
unk_bruteforcer,"[0.9411764705882353, 0.945054945054945, 0.9450...","[0.9132075471698113, 0.9213483146067416, 0.913...","[0.9022556390977444, 0.9022556390977444, 0.902..."
unk_exploiter,"[0.888888888888889, 0.888888888888889, 0.88888...","[0.7499999999999999, 0.7499999999999999, 0.749...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
unk_spammer,"[0.3846153846153846, 0.4444444444444445, 0.444...","[0.6451612903225806, 0.6451612903225806, 0.687...","[0.5806451612903225, 0.5806451612903225, 0.580..."
intrinsec,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
internetcensus,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [33]:
def mean(values):
    return np.trunc(np.mean(values) * 100) / 100
    
result_df = scores.applymap(mean)
macro = result_df.sort_index()
macro

,igcngru_features,idarkvec,features
censys,0.61,0.73,0.65
driftnet,0.29,0.92,0.75
internetcensus,0.20,0.33,0.00
intrinsec,0.06,0.66,0.00
ipip,0.44,0.11,0.00
mirai,0.97,0.97,0.96
onyphe,0.17,0.94,0.51
rapid7,0.64,1.00,0.91
shodan,0.00,0.00,0.00
u_mich,0.10,1.00,0.00


In [35]:
for col in prob_cols:
    if col in macro.index:
        print(';'.join([ str(v) for v in macro.loc[col].values]))
    else:
        print("0.0;0.0;0.0")
    

0.61;0.73;0.65
0.29;0.92;0.75
0.2;0.33;0.0
0.06;0.66;0.0
0.44;0.11;0.0
0.97;0.97;0.96
0.17;0.94;0.51
0.64;1.0;0.91
0.0;0.0;0.0
0.0;0.0;0.0
0.0;0.0;0.0
0.1;1.0;0.0
0.92;0.91;0.91
0.76;0.78;0.92
0.55;0.76;0.64
0.0;0.0;0.0
